<a href="https://colab.research.google.com/github/moze90/Thesis2022/blob/main/Thesis_2022_Newdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
import warnings
warnings.filterwarnings("ignore")
from glob import glob
import re
import gc
import csv
from scipy.stats import zscore
from datetime import datetime
from pandas.io.parsers.readers import read_csv
from sklearn.covariance import EllipticEnvelope
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import metrics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**DATA PREPROCESSING**

# **new EQ**

**First step:removing null values**

In [ ]:
df1=(r"/content/drive/MyDrive/Thesis_datasets/New-data-SoftPro/Hospitals1/eq.csv")

In [ ]:
 data1= pd.read_csv(df1,sep=';',header=0,parse_dates=True)

In [ ]:
data1.head(1)

,InvNr,InkDat,Skrotad,flReplacementCost,flPurchaseCost,SkrotDatum,ID,PMLMaintenanceRequirementID,PMLMissionCriticalID,PMLRiscID,...,Modell,Handelsnamn,intRisktal,strRiscName,intRiscSeverityLevel,FUSenaste,FUIntNasta,PMLMaintenanceRequirementDescription,PMLMissionCriticalDescription,PMLRiscDescription
0,C951-01,2001-01-01 00:00:00.000,2,0,33210,NaN,68263,NaN,NaN,NaN,...,Wolf 5512,NaN,20.0,20,20.0,2004-06-15 00:00:00.000,NaN,NaN,NaN,NaN


In [ ]:
data1.isnull().sum()

InvNr                                     2344
InkDat                                   13883
Skrotad                                      0
flReplacementCost                        65477
flPurchaseCost                            6997
SkrotDatum                              129351
ID                                           0
PMLMaintenanceRequirementID             226918
PMLMissionCriticalID                    226943
PMLRiscID                               226946
Benamn                                    2025
Fabrikat                                  2099
Modell                                    2150
Handelsnamn                             224409
intRisktal                                9497
strRiscName                               9497
intRiscSeverityLevel                      9497
FUSenaste                               146508
FUIntNasta                              178383
PMLMaintenanceRequirementDescription    226918
PMLMissionCriticalDescription           226943
PMLRiscDescri

In [ ]:
fin=data1.drop(columns=['intRiscSeverityLevel','Skrotad','flReplacementCost','flPurchaseCost','PMLMaintenanceRequirementID','PMLMissionCriticalID','PMLRiscID','strRiscName','Handelsnamn','PMLMaintenanceRequirementDescription','PMLMissionCriticalDescription','PMLRiscDescription','FUSenaste','FUIntNasta'])

In [ ]:
fin.head(4)

,InvNr,InkDat,SkrotDatum,ID,Benamn,Fabrikat,Modell,intRisktal
0,C951-01,2001-01-01 00:00:00.000,NaN,68263,Endoskop,Wolf,Wolf 5512,20.0
1,C951-02,2001-02-08 00:00:00.000,2018-12-19 00:00:00.000,68264,Endoskop,Wolf,Wolf 5512,31.0
2,111444,2005-01-01 00:00:00.000,NaN,86743,Endoskop,Wolf,Wolf 5512,20.0
3,9013559,2008-01-01 00:00:00.000,NaN,140398,Ljuskälla f endoskop,Wolf,4015,13.0


In [ ]:
dict = {'FUSenaste': 'prevPm',
        'FUIntNasta': 'nexPm',
        'InkDat':'Equip.reg_date',
        'SkrotDatum':'Scrapped_date',
        'Benamn':'dev',
      'Fabrikat':'Brand',
        'Modell':'Model',
        'intRisktal':'calc.Risk'}

In [ ]:
fin.rename(columns=dict,inplace=True)

In [ ]:
len(fin)

227003

**substitute scrapped null values with data collection**

In [ ]:
fin.isnull().sum()

InvNr               2344
Equip.reg_date     13883
Scrapped_date     129351
ID                     0
dev                 2025
Brand               2099
Model               2150
calc.Risk           9497
dtype: int64

In [ ]:
dict1 = {'InkDat': 'Equip.reg_date',
        'SkrotDatum': 'Scrapped_date',
        'Benamn':'dev',
        'Fabrikat':'Brand',
        'Modell':'Model',
        'intRisktal':'cal.Risk'}

In [ ]:
fin.head(3)

,InvNr,Equip.reg_date,Scrapped_date,ID,dev,Brand,Model,calc.Risk
0,C951-01,2001-01-01 00:00:00.000,NaN,68263,Endoskop,Wolf,Wolf 5512,20.0
1,C951-02,2001-02-08 00:00:00.000,2018-12-19 00:00:00.000,68264,Endoskop,Wolf,Wolf 5512,31.0
2,111444,2005-01-01 00:00:00.000,NaN,86743,Endoskop,Wolf,Wolf 5512,20.0


In [ ]:
fin.dropna(inplace=True)

In [ ]:
fin.shape

(83584, 8)

In [ ]:
fin.set_index('ID',inplace=True,drop=False)

In [ ]:
fin.head(3)

,InvNr,Equip.reg_date,Scrapped_date,ID,dev,Brand,Model,calc.Risk
ID,,,,,,,,
68264,C951-02,2001-02-08 00:00:00.000,2018-12-19 00:00:00.000,68264,Endoskop,Wolf,Wolf 5512,31.0
112347,P384,1981-01-01 00:00:00.000,2012-07-19 00:00:00.000,112347,Ljuskälla f endoskop,Wolf,4015,31.0
88325,C0297,1985-01-01 00:00:00.000,2015-12-16 00:00:00.000,88325,Ljuskälla f endoskop,Wolf,4015,31.0


# ***pm***

In [ ]:
df3=(r"/content/drive/MyDrive/Thesis_datasets/New-data-SoftPro/Hospitals1/pm.csv")

In [ ]:
fin3=read_csv(df3,sep=';')

In [ ]:
fin3.head(3)

,Id,EquipmentId,NextPM,LastPM,Interval,IsRecurring
0,7838,17635.0,2007-01-01 00:00:00.000,2006-01-12 00:00:00.000,12 Månader,1
1,27837,65129.0,2003-06-15 00:00:00.000,2001-06-15 00:00:00.000,24 Månader,1
2,27838,65130.0,2007-07-01 00:00:00.000,2005-07-25 00:00:00.000,24 Månader,1


In [ ]:
fin3.isnull().sum()

Id                 0
EquipmentId     4081
NextPM          4921
LastPM         47521
Interval       13329
IsRecurring        0
dtype: int64

In [ ]:
len(fin3)

117291

In [ ]:
fin3.dropna(inplace=True)

In [ ]:
len(fin3)

68762

In [ ]:
fin3.isnull().sum()

Id             0
EquipmentId    0
NextPM         0
LastPM         0
Interval       0
IsRecurring    0
dtype: int64

In [ ]:
fin3.count()

Id             68762
EquipmentId    68762
NextPM         68762
LastPM         68762
Interval       68762
IsRecurring    68762
dtype: int64

In [ ]:
fin3.set_index('EquipmentId',drop=False)

,Id,EquipmentId,NextPM,LastPM,Interval,IsRecurring
EquipmentId,,,,,,
17635.0,7838,17635.0,2007-01-01 00:00:00.000,2006-01-12 00:00:00.000,12 Månader,1
65129.0,27837,65129.0,2003-06-15 00:00:00.000,2001-06-15 00:00:00.000,24 Månader,1
65130.0,27838,65130.0,2007-07-01 00:00:00.000,2005-07-25 00:00:00.000,24 Månader,1
65131.0,27839,65131.0,2006-06-01 00:00:00.000,2004-06-01 00:00:00.000,24 Månader,1
65132.0,27840,65132.0,2003-06-26 00:00:00.000,2001-06-26 00:00:00.000,24 Månader,1
...,...,...,...,...,...,...
305222.0,253852,305222.0,2022-09-01 00:00:00.000,2021-09-01 00:00:00.000,12 Månader,1
305223.0,253853,305223.0,2022-09-01 00:00:00.000,2021-09-01 00:00:00.000,12 Månader,1
305224.0,253854,305224.0,2022-09-01 00:00:00.000,2021-09-01 00:00:00.000,12 Månader,1


# **pm data**

In [ ]:
df2=(r"/content/drive/MyDrive/Thesis_datasets/New-data-SoftPro/Hospitals1/pm_data.csv")

In [ ]:
fin1=read_csv(df2,sep=';',names=['invID','last_pm','first_pm','count_pm','meantime_pm'])

In [ ]:
len(fin1)

80503

In [ ]:
fin1.head(3)

,invID,last_pm,first_pm,count_pm,meantime_pm
0,204282,2020-03-04,2009-03-31,11,362
1,255222,2021-06-28,2018-03-20,4,299
2,208527,2015-06-22,2015-06-22,1,0


In [ ]:
fin1.isnull().sum()

invID          0
last_pm        0
first_pm       0
count_pm       0
meantime_pm    0
dtype: int64

In [ ]:
fin1.isnull().sum()

invID          0
last_pm        0
first_pm       0
count_pm       0
meantime_pm    0
dtype: int64

In [ ]:
fin1.count()

invID          80503
last_pm        80503
first_pm       80503
count_pm       80503
meantime_pm    80503
dtype: int64

In [ ]:
fin1.set_index('invID',drop=False,inplace=True)

In [ ]:
fin1.head(3)

,invID,last_pm,first_pm,count_pm,meantime_pm
invID,,,,,
204282,204282,2020-03-04,2009-03-31,11,362
255222,255222,2021-06-28,2018-03-20,4,299
208527,208527,2015-06-22,2015-06-22,1,0



# pm **history**

In [ ]:
df4=(r"/content/drive/MyDrive/Thesis_datasets/New-data-SoftPro/Hospitals1/pm_history.csv")

In [ ]:
fin4=read_csv(df4,sep=';')

In [ ]:
fin4.head(3)

In [ ]:
fin4.shape

In [ ]:
fin4.isnull().sum()

we need to study devices which have recurring pm activities

In [ ]:
fin4['blnRecurring'].value_counts()

In [ ]:
recurring_hos1=fin4.loc[fin4['blnRecurring']== 1]

In [ ]:
recurring_hos1.shape

now that we have got only the devices with recurring pm activities , we need to drop all irrelevent columns before translating the rest of the columns 

In [ ]:
recurring_hos1.drop(columns=['blnRecurring','intTimeIntervalRecommended','intEstimatedPMTimeHours','(No column name).1','(No column name).2','blnHiddenPMClock'],inplace=True)

In [ ]:
recurring_hos1

# WO

In [ ]:
df6=(r"/content/drive/MyDrive/Thesis_datasets/New-data-SoftPro/Hospitals1/wo.csv")

In [ ]:
dic={'ID':'ID','AORegDat':'issued','AOKlarDat':'Completed','AOEKlarDat':'fin_com','InventarieID':'InventarieID'}

In [ ]:
fin6=read_csv(df6,sep=';')

In [ ]:
fin6.rename(columns=dic,inplace=True)

In [ ]:
fin6.query("InventarieID==	220059.0")

In [ ]:
fin6.head(10)

In [ ]:
fin6.isnull().sum()

In [ ]:
fin6.dropna(inplace=True)

In [ ]:
fin6.shape

In [ ]:
fin6.set_index('InventarieID',inplace=True,drop=False)

In [ ]:
fin6.head(3)

In [ ]:
#sorting the dataframe based on the inventarieID descedning order
fin6=fin6.sort_index()


In [ ]:
fin6.iloc[[101]]

# **wo data**

In [ ]:
df7=(r"/content/drive/MyDrive/Thesis_datasets/New-data-SoftPro/Hospitals1/WO_data.csv")

In [ ]:
fin7=pd.read_csv(df7,sep=';',names=['inventarieID','last_wo','first_wo','Count_wo','meantime'])


In [ ]:
fin7.head(3)

In [ ]:
fin7.dropna(inplace=True)

In [ ]:
fin7.shape

In [ ]:
fin7.set_index('inventarieID',inplace=True,drop=False)

In [ ]:
fin7.head(3)

# **Connecting recurring hospital pm history data with equipment data to query based on specific devices**

In [ ]:
recurring_hos1.set_index('InventarierID',inplace=True,drop=False)

In [ ]:
recurring_hos1.head(3)

,(No column name),ID,intTimeInterval,PM_TimeIntervalUnitID,PM_TimeIntervalUnitIDRecommended,InventarierID,ArborderID,DeviatingTimeIntervalRecommended,PM_diff
InventarierID,,,,,,,,,
65129,2011-04-03,27837,24.0,2.0,NaN,65129,NaN,1,3579
65375,2011-04-03,28001,24.0,2.0,NaN,65375,NaN,1,2573
65657,2011-04-03,28245,12.0,2.0,NaN,65657,NaN,1,2823


In [ ]:
pmhist_eq=pd.merge(fin,recurring_hos1,left_index=True,right_index=True)

In [ ]:
pmhist_eq.shape

(86612, 17)

In [ ]:
deviate_hos1=pmhist_eq.loc[pmhist_eq['DeviatingTimeIntervalRecommended'] == 1]

In [ ]:
deviate_hos1

,InvNr,Equip.reg_date,Scrapped_date,ID_x,dev,Brand,Model,calc.Risk,(No column name),ID_y,intTimeInterval,PM_TimeIntervalUnitID,PM_TimeIntervalUnitIDRecommended,InventarierID,ArborderID,DeviatingTimeIntervalRecommended,PM_diff
17500,9004181,2007-01-01 00:00:00.000,2013-11-13 00:00:00.000,17500,Lungröntgenlab,Fuji,Velocity,31.0,2011-02-28,7800,12.0,2.0,NaN,17500,NaN,1,4112
17635,R1273-01,1998-01-01 00:00:00.000,2009-12-15 00:00:00.000,17635,Undersökningsstativ/system,Siemens,Siemens Symphony,31.0,2011-02-28,7838,12.0,2.0,NaN,17635,NaN,1,1873
65129,C105-66,1991-01-01 00:00:00.000,2003-12-15 00:00:00.000,65129,Infusionspump,Cardinal Health,IVAC 561,31.0,2011-04-03,27837,24.0,2.0,NaN,65129,NaN,1,3579
65130,C105-67,1991-01-01 00:00:00.000,2006-04-03 00:00:00.000,65130,Infusionspump,Cardinal Health,IVAC 561,31.0,2011-04-03,27838,24.0,2.0,NaN,65130,NaN,1,2078
65131,C105-68,1991-01-01 00:00:00.000,2006-08-21 00:00:00.000,65131,Infusionspump,Cardinal Health,IVAC 561,31.0,2011-04-03,27839,24.0,2.0,NaN,65131,NaN,1,2497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252318,000887,2009-12-12 00:00:00.000,2012-08-29 00:00:00.000,252318,Projektor,LG,DX630,31.0,2012-02-16,1590757,6.0,2.0,NaN,252318,1160730.0,1,0
252318,000887,2009-12-12 00:00:00.000,2012-08-29 00:00:00.000,252318,Projektor,LG,DX630,31.0,2012-02-16,1590787,6.0,2.0,NaN,252318,1160731.0,1,0
252599,002523,2012-02-23 00:00:00.000,2016-11-07 00:00:00.000,252599,"Dator, bärbar/surfplatta (Medical grade)",Dell,Latitude E6320,31.0,2016-11-07,1590800,12.0,2.0,2.0,252599,1160873.0,1,0
252706,002531,2012-09-13 00:00:00.000,2021-05-31 00:00:00.000,252706,"Dator, bärbar/surfplatta (Medical grade)",Dell,Latitude E6230,31.0,2016-02-09,1590775,12.0,2.0,NaN,252706,1160880.0,1,0


In [ ]:
deviate_hos1.drop(columns=['DeviatingTimeIntervalRecommended'],inplace=True)

In [ ]:
deviate_hos1.shape

(75186, 16)

In [ ]:
non_deviatehos1=pmhist_eq.loc[pmhist_eq['DeviatingTimeIntervalRecommended']==0]

In [ ]:
non_deviatehos1

,InvNr,Equip.reg_date,Scrapped_date,ID_x,dev,Brand,Model,calc.Risk,(No column name),ID_y,intTimeInterval,PM_TimeIntervalUnitID,PM_TimeIntervalUnitIDRecommended,InventarierID,ArborderID,DeviatingTimeIntervalRecommended,PM_diff
39879,9004189,2008-01-01 00:00:00.000,2021-09-21 00:00:00.000,39879,"Ultraljudssystem, bildgivande universellt",Acuson,Sequoia 512,18.0,2021-03-05,220794,12.0,2.0,NaN,39879,1360473.0,0,0
41575,9045655,2011-01-01 00:00:00.000,2021-12-20 00:00:00.000,41575,"Röntgensystem för benabsorptionsmätning, dubbe...",Hologic,Discovery A,16.0,2021-09-06,216789,6.0,2.0,NaN,41575,1398455.0,0,33
41575,9045655,2011-01-01 00:00:00.000,2021-12-20 00:00:00.000,41575,"Röntgensystem för benabsorptionsmätning, dubbe...",Hologic,Discovery A,16.0,2020-08-13,216789,12.0,2.0,NaN,41575,1320087.0,0,0
41575,9045655,2011-01-01 00:00:00.000,2021-12-20 00:00:00.000,41575,"Röntgensystem för benabsorptionsmätning, dubbe...",Hologic,Discovery A,16.0,2020-05-05,216789,12.0,2.0,NaN,41575,1300740.0,0,266
41575,9045655,2011-01-01 00:00:00.000,2021-12-20 00:00:00.000,41575,"Röntgensystem för benabsorptionsmätning, dubbe...",Hologic,Discovery A,16.0,2021-02-22,216789,6.0,2.0,NaN,41575,1357755.0,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300077,1984-881,2019-08-14 00:00:00.000,2022-02-17 00:00:00.000,300077,Termometer öron IR,Braun,ThermoScan Pro 6000,13.0,2022-02-17,236157,12.0,2.0,2.0,300077,1426779.0,0,0
301900,351114,2001-03-04 00:00:00.000,2021-10-05 00:00:00.000,301900,Patientsäng el,Proton Caretec,10101,6.0,2021-07-13,241464,12.0,2.0,NaN,301900,1387727.0,0,0
301957,351171,2008-04-24 00:00:00.000,2021-10-05 00:00:00.000,301957,Patientsäng el,Proton Caretec,17603,6.0,2021-08-06,241122,12.0,2.0,NaN,301957,1392348.0,0,69
301957,351171,2008-04-24 00:00:00.000,2021-10-05 00:00:00.000,301957,Patientsäng el,Proton Caretec,17603,6.0,2021-08-17,241122,12.0,2.0,NaN,301957,1394217.0,0,81


In [ ]:
non_deviatehos1.shape

(11426, 17)

query our usual devices from those lists of deviated and non-deviated 

**patient monitor**

In [ ]:
dev_pa=deviate_hos1.query("dev == 'Patientmonitor multiparameter'")

In [ ]:
dev_pa.shape

(6053, 16)

In [ ]:
nondev_pa=non_deviatehos1.query("dev =='patientmonitor multiparameter'")

In [ ]:
nondev_pa.shape

(0, 17)

**anathesai**

In [ ]:
data50=deviate_hos1.query("dev == 'Anestesiarbetsstation'")
data30=ata2=deviate_hos1.query("dev == 'Anestesiarbetsstation'")
data35=deviate_hos1.query("dev == 'Modul till anestesiarbetsstation'")
data40=deviate_hos1.query("dev == 'Anestesimedelsförgasare, isofluran'")
data55=deviate_hos1.query("dev == 'Anestesimedel Förgasare'")
data60=deviate_hos1.query("dev == 'Anestesiapparat mobil'")
data70=deviate_hos1.query("dev == 'Lungventilator anestesi'")
deviatedata_anes = pd.concat([data50, data30,data35,data50, data40, data55, data60, data70], axis=0)

In [ ]:
deviatedata_anes.shape

(3050, 16)

In [ ]:
data51=non_deviatehos1.query("dev == 'Anestesiarbetsstation'")
data31=ata2=non_deviatehos1.query("dev == 'Anestesiarbetsstation'")
data36=deviate_hos1.query("dev == 'Modul till anestesiarbetsstation'")
data41=deviate_hos1.query("dev == 'Anestesimedelsförgasare, isofluran'")
data56=deviate_hos1.query("dev == 'Anestesimedel Förgasare'")
data61=deviate_hos1.query("dev == 'Anestesiapparat mobil'")
data71=deviate_hos1.query("dev == 'Lungventilator anestesi'")
non_deviatedata_anes = pd.concat([data51, data31,data36,data51, data41, data56, data61, data71], axis=0)

In [ ]:
non_deviatedata_anes.shape

(1247, 17)

**pm overdue**

In [ ]:
df5=(r"/content/drive/MyDrive/Thesis_datasets/New-data-SoftPro/Hospitals1/PM_overdue.csv")

In [ ]:
fin5=read_csv(df5,sep=';',names=['inID','V1','V2','V3'])

In [ ]:
fin5.head(3)

,inID,V1,V2,V3
0,89667,4,0,1
1,140607,6,0,0
2,93912,5,0,1


# Connecting **wo** AND **WO_data** files based on inventarieID 


In [ ]:
inner_merged_total_WO = pd.merge(fin6,fin7,left_index=True, right_index=True)

In [ ]:
inner_merged_total_WO.shape

(707677, 10)

In [ ]:
inner_merged_total_WO.head(10)

,ID,issued,Completed,AOEkKlarDat,InventarieID,inventarieID,last_wo,first_wo,Count_wo,meantime
1364.0,1233471,2019-05-03 00:00:00.000,2019-05-09 00:00:00.000,2019-05-09 00:00:00.000,1364.0,1364.0,2020-08-13,2011-03-25,65,52
1364.0,662950,2017-05-26 00:00:00.000,2017-05-26 00:00:00.000,2017-05-26 00:00:00.000,1364.0,1364.0,2020-08-13,2011-03-25,65,52
1364.0,612067,2016-01-04 00:00:00.000,2016-01-04 00:00:00.000,2016-01-04 00:00:00.000,1364.0,1364.0,2020-08-13,2011-03-25,65,52
1364.0,393467,2012-07-23 00:00:00.000,2012-12-21 00:00:00.000,2012-12-21 00:00:00.000,1364.0,1364.0,2020-08-13,2011-03-25,65,52
1364.0,1237176,2019-05-21 00:00:00.000,2019-05-21 00:00:00.000,2019-05-21 00:00:00.000,1364.0,1364.0,2020-08-13,2011-03-25,65,52
1364.0,1222558,2019-02-28 00:00:00.000,2019-02-28 00:00:00.000,2019-02-28 00:00:00.000,1364.0,1364.0,2020-08-13,2011-03-25,65,52
1364.0,1167578,2018-04-24 00:00:00.000,2018-04-24 00:00:00.000,2018-04-24 00:00:00.000,1364.0,1364.0,2020-08-13,2011-03-25,65,52
1364.0,510206,2013-02-14 00:00:00.000,2013-02-14 00:00:00.000,2013-02-14 00:00:00.000,1364.0,1364.0,2020-08-13,2011-03-25,65,52
1364.0,397564,2012-10-09 00:00:00.000,2012-10-09 00:00:00.000,2012-10-09 00:00:00.000,1364.0,1364.0,2020-08-13,2011-03-25,65,52
1364.0,394949,2012-08-24 00:00:00.000,2013-01-28 00:00:00.000,2013-01-28 00:00:00.000,1364.0,1364.0,2020-08-13,2011-03-25,65,52


In [ ]:
inner_merged_total_WO.value_counts()

In [ ]:
test1=inner_merged_total_WO.query("inventarieID==92123.0")
test1.shape
test1

In [ ]:
inner_merged_total_WO.count()

# **Connecting work orders data with equipment data first **

In [ ]:
alldata_wo_eq=pd.merge(fin,inner_merged_total_WO,left_index=True,right_index=True)

In [ ]:
alldata_wo_eq.shape

In [ ]:
tes=alldata_wo_eq.query("ID_x == 1364")

In [ ]:
tes

In [ ]:
alldata_wo_eq.drop(columns=['InvNr','Brand','Model','AOEkKlarDat','Completed','last_wo','first_wo','ID_x'],inplace=True)

In [ ]:
alldata_wo_eq.head(3)

**Obtain Age column**

In [ ]:
alldata_wo_eq['Equip.reg_date']=pd.to_datetime(alldata_wo_eq['Equip.reg_date'])
alldata_wo_eq['Scrapped_date']=pd.to_datetime(alldata_wo_eq['Scrapped_date'])



In [ ]:
alldata_wo_eq['age']=(alldata_wo_eq['Scrapped_date']-alldata_wo_eq['Equip.reg_date']).dt.days


In [ ]:
alldata_wo_eq.drop(columns=['Scrapped_date','InventarieID'],inplace=True)

In [ ]:
alldata_wo_eq.head(3)

In [ ]:
# we have got 1782 registered devices within our data set
alldata_wo_eq.dev.nunique()

In [ ]:
#checking the highest device with the highest no of samples 
alldata_wo_eq.dev.value_counts()

In [ ]:
#devices counter
dev_count=alldata_wo_eq.dev.value_counts()

In [ ]:
dev_count.head(20)

Query different devices

oxygen concentrators

In [ ]:
dataoxyhos1=alldata_wo_eq.query("dev == 'Oxygenkoncentrator stationär'")



In [ ]:
dataoxyhos1.head(3)

anathesia machines

In [ ]:
data22=alldata_wo_eq.query("dev == 'Anestesiarbetsstation'")
data32=alldata_wo_eq.query("dev == 'Modul till anestesiarbetsstation'")
data42=alldata_wo_eq.query("dev == 'Anestesimedelsförgasare, isofluran'")
data52=alldata_wo_eq.query("dev == 'Anestesimedel Förgasare'")
data62=alldata_wo_eq.query("dev == 'Anestesiapparat mobil'")
data72=alldata_wo_eq.query("dev == 'Lungventilator anestesi'")
data_aneshos1woeq = pd.concat([data22, data32, data42, data52, data62, data72], axis=0)
data_aneshos1woeq.head(3)


In [ ]:
data_aneshos1woeq['issued']=pd.to_datetime(data_aneshos1woeq['issued'])

In [ ]:
data_aneshos1woeq.shape

**plotting data points**

In [ ]:
data_aneshos1woeq.plot(y='ID_y',x='issued',kind='scatter',title='workorder versus dates of issuance')

In [ ]:
data_aneshos1woeq.plot(y='age',x='issued',kind='scatter',title='age with workorders')

In [ ]:
data_aneshos1woeq.plot(x='age',y='Count_wo',kind='scatter',title='age with workorders')

# **Connecting pm and wo with equipments(all data together)**

bringing eq data and setting ID field as an index to prepare for merge

In [ ]:
fin.shape

In [ ]:
eq=fin.set_index('ID',drop=False)

In [ ]:
eq.head(3)

### **CONNECTING PM-data With WO COMBINED DATA** **bold text**

In [ ]:
inner_mergeWO_pm = pd.merge(inner_merged_total_WO,fin1,left_index=True, right_index=True)

In [ ]:
all_data_together= pd.merge(eq,inner_mergeWO_pm,left_index=True, right_index=True)

In [ ]:
all_data_together.head(10)

sorting based on inventariedID and issued **date**

In [ ]:
all_data_together=all_data_together.sort_values(by=['inventarieID','issued'])

In [ ]:
all_data_together=all_data_together.drop(columns=['InvNr','Completed','InventarieID','invID'])

In [ ]:
all_data_together.shape

In [ ]:
all_data_together.head(10)

# **Multiple Regression model for Work Orders and Failing time -Anathesia device**

In [ ]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing

In [ ]:
data_aneshos1woeq.shape

model pre-processing

table find difference between fault date and register date 

In [ ]:
data_aneshos1woeq['wotime_diff']=(data_aneshos1woeq['issued']-data_aneshos1woeq['Equip.reg_date']).dt.days

In [ ]:
data_aneshos1woeq.tail(15)

it is not practical for the model to predict certain date and day , it more commonly practices that there is a time window for the decision to be taken .

 **Exploratory data analysis**

Anathesia devices

In [ ]:
data_aneshos1woeq.head(3)

In [ ]:
#plotting age
data_aneshos1woeq.plot(x='Count_wo',y='wotime_diff',kind='scatter')

In [ ]:
sns.pairplot(data_aneshos1woeq.corr())

In [ ]:
Y_reghos1=data_aneshos1woeq['wotime_diff']
X_reghos1=data_aneshos1woeq.drop(['dev','ID_y','issued','Equip.reg_date','wotime_diff','inventarieID'],axis=1)


In [ ]:
Y_reghos1.head(3)

In [ ]:
X_train_aneshos1, X_test_aneshos1, y_train_aneshos1, y_test_aneshos1 = train_test_split(
    X_reghos1, Y_reghos1, test_size=0.3, random_state=101)

In [ ]:
y_test_aneshos1.head(3)

In [ ]:
# creating a regression model
model_anes = LinearRegression()


In [ ]:
model_anes.fit(X_train_aneshos1,y_train_aneshos1)

In [ ]:
predictions_anes= model_anes.predict(X_test_aneshos1)


In [ ]:
 predictions_anes

In [ ]:
predictions_anes.shape

In [ ]:
predictions_anes_df=pd.DataFrame(predictions_anes)

In [ ]:
age_col=pd.DataFrame(X_reghos1['age'])
age_col=age_col.reset_index()

In [ ]:
age_col.head(3)

In [ ]:
pred_fin=pd.concat([predictions_anes_df,age_col],axis=1)

In [ ]:
pred_fin.drop(columns=['index'],axis=1)

In [ ]:
pred_fin.columns = ['index','wo_timeprednonorm','age']


In [ ]:
pred_fin.head(3)

In [ ]:
# model evaluation
print(
'mean_squared_error : ', mean_squared_error(y_test_aneshos1, predictions_anes_df))
print(
'mean_absolute_error : ', mean_absolute_error(y_test_aneshos1, predictions_anes_df))


# **EDA2**

In [ ]:
#it is a lot better to take one or two devices as samples to see the workrder patterns

indexed_MD=data_aneshos1woeq.loc[231344]

In [ ]:
indexed_MD

In [ ]:
indexed_MD.set_index('issued',inplace=True,drop=False)

In [ ]:
indexed_MD.index

**Trying normalization and see if it works**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler=MinMaxScaler()
scaler.fit(X_reghos1)
scaled=scaler.fit_transform(X_reghos1)
scaled_df=pd.DataFrame(scaled,columns=X_reghos1.columns)

In [ ]:
scaled_df.shape

In [ ]:
scaled_df.head(3)


In [ ]:
X_train_aneshos1norm, X_test_aneshos1norm, y_train_aneshos1norm, y_test_aneshos1norm = train_test_split(
    scaled_df, Y_reghos1,test_size=0.3, random_state=101)

In [ ]:
Y_reghos1.head(3)

In [ ]:
model_anesnorm = LinearRegression()


In [ ]:
model_anesnorm.fit(X_train_aneshos1norm,y_train_aneshos1norm)

In [ ]:
predictions_anesnorm= model_anesnorm.predict(X_test_aneshos1norm)


In [ ]:
predictions_anesnorm

In [ ]:
predictions_anes_dfnorm=pd.DataFrame(predictions_anesnorm)

In [ ]:
predictions_anes_dfnorm.head(3)

In [ ]:
predictions_anes_dfnorm.columns = ['wo_timepred']


In [ ]:
predictions_anes_dfnorm.head(3)

In [ ]:
# model evaluation
print(
'mean_squared_error : ', mean_squared_error(y_test_aneshos1, predictions_anes_dfnorm))
print(
'mean_absolute_error : ', mean_absolute_error(y_test_aneshos1, predictions_anes_dfnorm))


# **OXYGEN CONCENTRATOR Model after adding more adding pm attributes**

In [ ]:
pm_wo_attributes_oxygen=pd.merge(dataoxyhos1,fin1,left_index=True,right_index=True)

In [ ]:
pm_wo_attributes_oxygen.head(3)

In [ ]:
pm_wo_attributes_oxygen.shape

In [ ]:
pm_wo_attributes_oxygen['Equip.reg_date']=pd.to_datetime(pm_wo_attributes_oxygen['Equip.reg_date'])
pm_wo_attributes_oxygen['issued']=pd.to_datetime(pm_wo_attributes_oxygen['issued'])


In [ ]:
pm_wo_attributes_oxygen['wotime_diff']=(pm_wo_attributes_oxygen['issued']-pm_wo_attributes_oxygen['Equip.reg_date']).dt.days

In [ ]:
pm_wo_attributes_oxygen.drop(columns=['Equip.reg_date','issued','invID','last_pm','first_pm','calc.Risk'],inplace=True)

# **Regression model for oxygen concentrators**

In [ ]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing

In [ ]:
dataoxyhos1.shape

In [ ]:
dataoxyhos1.nunique()

In [ ]:
dataoxyhos1['Equip.reg_date']=pd.to_datetime(dataoxyhos1['Equip.reg_date'])
dataoxyhos1['issued']=pd.to_datetime(dataoxyhos1['issued'])



In [ ]:
dataoxyhos1.drop(columns=['dev','inventarieID','ID_y'],inplace=True)

In [ ]:
dataoxyhos1['wotime_diff']=(dataoxyhos1['issued']-dataoxyhos1['Equip.reg_date']).dt.days

In [ ]:
dataoxyhos1.corr()

In [ ]:
dataoxyhos1.describe()

In [ ]:
Y_oxygen=dataoxyhos1['wotime_diff']
X_oxygen=dataoxyhos1.drop(['issued','Equip.reg_date','wotime_diff'],axis=1)


In [ ]:
X_train_oxy1, X_test_oxy1, y_train_oxy1, y_test_oxy1 = train_test_split(X_oxygen,Y_oxygen, test_size=0.3, random_state=101)

In [ ]:
y_train_oxy1.head(3)

In [ ]:
model_oxy = LinearRegression()
model_oxy.fit(X_train_oxy1,y_train_oxy1)

In [ ]:
predictions_oxy=model_oxy.predict(X_test_oxy1)

In [ ]:
# model evaluation
print(
'mean_squared_error : ', mean_squared_error(y_test_oxy1, predictions_oxy))
print(
'mean_absolute_error : ', mean_absolute_error(y_test_oxy1, predictions_oxy))


# **Patient_monitor experiments**

In [ ]:
patient_monitor=all_data_together.query("dev == 'Patientmonitor multiparameter'")


In [ ]:
patient_monitor.head(3)

In [ ]:
patient_monitor.shape

In [ ]:
refined_patient_monitor=patient_monitor.drop(columns=['Brand','ID_x','dev','Brand','calc.Risk', 'ID_y','last_wo','first_wo'])

In [ ]:
refined_patient_monitor.head(3)

In [ ]:
refined_patient_monitor['Equip.reg_date']=pd.to_datetime(refined_patient_monitor['Equip.reg_date'])
refined_patient_monitor['issued']=pd.to_datetime(refined_patient_monitor['issued'])
refined_patient_monitor['Scrapped_date']=pd.to_datetime(refined_patient_monitor['Scrapped_date'])




In [ ]:
refined_patient_monitor['wotime_diff']=(refined_patient_monitor['issued']-refined_patient_monitor['Equip.reg_date']).dt.days
refined_patient_monitor['age']=(refined_patient_monitor['Scrapped_date']-refined_patient_monitor['Equip.reg_date']).dt.days

In [ ]:
refined_patient_monitor.head(3)

In [ ]:
refined_patient_monitor.shape

In [ ]:
#taking out outliers 
refined_patient_monitor.plot(x='Equip.reg_date',y='Scrapped_date',kind='scatter')


In [ ]:
no_outlier_pamonitor=refined_patient_monitor.loc[refined_patient_monitor['wotime_diff']>=1]

In [ ]:
no_outlier_pamonitor.corr()

In [ ]:
no_outlier_pamonitor.shape

In [ ]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing

In [ ]:
X_PA=no_outlier_pamonitor.drop(['Equip.reg_date','Scrapped_date','issued','AOEkKlarDat','Model','last_pm','first_pm','inventarieID'],axis=1)
Y_PA=no_outlier_pamonitor['wotime_diff']

In [ ]:
Y_PA.head(10)

In [ ]:
X_train_pamonitor, X_test_pamonitor, y_train_pamonitor, y_test_pamonitor = train_test_split(X_PA,Y_PA, test_size=0.3, random_state=101)

In [ ]:
X_train_pamonitor.head()

In [ ]:
# creating a regression model
model_PA = LinearRegression()


In [ ]:
model_PA.fit(X_train_pamonitor,y_train_pamonitor)

In [ ]:
predictions_PA= model_PA.predict(X_test_pamonitor)


In [ ]:
predictions_PA=pd.DataFrame(predictions_PA)

In [ ]:
predictions_PA.head(10)

In [ ]:
y_test_pamonitor.head(10)

In [ ]:
# model evaluation
print(
'mean_squared_error : ', mean_squared_error(y_test_pamonitor, predictions_PA))
print(
'mean_absolute_error : ', mean_absolute_error(y_test_pamonitor, predictions_PA))


# **Chosen devices in one regression model experiment**

In [ ]:
#after combining wo data with preventive mainteance data in one dataframe,we want to query the chosen device of anathesia and oxygen concentrators:
all_data_together.head(3)

In [ ]:
refined_all_data=all_data_together.drop(['ID_x','AOEkKlarDat','inventarieID','last_wo','first_wo','last_pm','first_pm'],axis=1)

In [ ]:
refined_all_data.head(3)

In [ ]:
data157=refined_all_data.query("dev == 'Anestesiarbetsstation'")
data156=refined_all_data.query("dev == 'Anestesiarbetsstation'")
data155=refined_all_data.query("dev == 'Modul till anestesiarbetsstation'")
data154=refined_all_data.query("dev == 'Anestesimedelsförgasare, isofluran'")
data153=refined_all_data.query("dev == 'Anestesimedel Förgasare'")
data152=refined_all_data.query("dev == 'Anestesiapparat mobil'")
data151=refined_all_data.query("dev == 'Lungventilator anestesi'")
data_anesWo_pm = pd.concat([data157, data156,data155,data154, data153, data152, data151], axis=0)

In [ ]:
data_anesWo_pm.shape

In [ ]:
#query oxygen concentrators
data_oxy_wo_pm=refined_all_data.query("dev == 'Oxygenkoncentrator stationär'")

In [ ]:
data_oxy_wo_pm.shape

In [ ]:
oxyplusana_wo_pm=pd.concat([data_oxy_wo_pm,data_anesWo_pm], axis=0)

In [ ]:
oxyplusana_wo_pm.shape

In [ ]:
oxyplusana_wo_pm.head(3)

In [ ]:
#preprocessing of these data before merge with patient monitor data.
oxyplusana_wo_pm['Equip.reg_date']=pd.to_datetime(oxyplusana_wo_pm['Equip.reg_date'])
oxyplusana_wo_pm['Scrapped_date']=pd.to_datetime(oxyplusana_wo_pm['Scrapped_date'])
oxyplusana_wo_pm['issued']=pd.to_datetime(oxyplusana_wo_pm['issued'])

In [ ]:
oxyplusana_wo_pm['wotime_diff']=(oxyplusana_wo_pm['issued']-oxyplusana_wo_pm['Equip.reg_date']).dt.days

In [ ]:
oxyplusana_wo_pm['age']=(oxyplusana_wo_pm['Scrapped_date']-oxyplusana_wo_pm['Equip.reg_date']).dt.days


In [ ]:
oxyplusana_wo_pm.drop(['Equip.reg_date','Scrapped_date','dev','Brand','Model','calc.Risk','ID_y','issued'],axis=1,inplace=True)

In [ ]:
oxyplusana_wo_pm.head(3)

In [ ]:
X_PA.head(3)

In [ ]:
#concat all devices(anathesia,oxygenconcentrator and patient monitor)
alldevices_regression=pd.concat((oxyplusana_wo_pm,X_PA),axis=0)

In [ ]:
x_alla=alldevices_regression.drop(['wotime_diff'],axis=1)
y_alla=alldevices_regression.drop(['Count_wo','meantime','count_pm','meantime_pm','age'],axis=1)
  

In [ ]:
y_alla.head(3)

In [ ]:
x_alla_training,x_alla_test,y_alla_training,y_alla_test=train_test_split(x_alla,y_alla,test_size=0.3,random_state=101)

In [ ]:
x_alla_training.shape

In [ ]:
y_alla_training.shape

In [ ]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing

In [ ]:
#now we apply the model 
regress_alldevices=LinearRegression()

In [ ]:
modelen=regress_alldevices.fit(x_alla_training,y_alla_training)

In [ ]:
predictions_alla=modelen.predict(x_alla_test)

In [ ]:
# model evaluation
print(
'mean_squared_error : ', mean_squared_error(y_alla_test, predictions_alla))
print(
'mean_absolute_error : ', mean_absolute_error(y_alla_test, predictions_alla))


# **#  Ridge regression for regression_patient monitor**

In [ ]:
import pandas as pd
from numpy import arange
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold

In [ ]:
X_PA=no_outlier_pamonitor.drop(['Equip.reg_date','Scrapped_date','issued','AOEkKlarDat','Model','last_pm','first_pm','inventarieID'],axis=1)
Y_PA=no_outlier_pamonitor['wotime_diff']

In [ ]:
X_train_pamonitor_ridge, X_test_pamonitor_ridge, y_train_pamonitor_ridge, y_test_pamonitor_ridge = train_test_split(X_PA,Y_PA, test_size=0.3, random_state=101)

In [ ]:
X_train_pamonitor_ridge.head(3)

In [ ]:
#define cross-validation method to evaluate model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [ ]:
#define model
model_ridge = RidgeCV(alphas=arange(0, 0.5, 0.01), cv=cv, scoring='neg_mean_absolute_error')

In [ ]:
#fit model
model_ridge.fit(X_train_pamonitor_ridge,y_train_pamonitor_ridge)

#display lambda that produced the lowest test MSE
print(model_ridge.alpha_)

In [ ]:
predictions_paridge=model_ridge.predict(X_test_pamonitor_ridge)

In [ ]:
# model evaluation
print(
'mean_squared_error : ', mean_squared_error(y_test_pamonitor,predictions_paridge))
print(
'mean_absolute_error : ', mean_absolute_error(y_test_pamonitor_ridge,predictions_paridge))


In [ ]:
predictions_paridge

In [ ]:
y_test_pamonitor

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso = Lasso(alpha=1.0)

In [ ]:
lasso.fit(X_train_pamonitor_ridge,y_train_pamonitor_ridge)


In [ ]:
lasso.score(X_test_pamonitor_ridge,y_test_pamonitor_ridge), lasso.score(X_train_pamonitor_ridge,y_train_pamonitor_ridge)

In [ ]:
predictions_lasso=lasso.predict(X_test_pamonitor_ridge)

In [ ]:
# model evaluation
print(
'mean_squared_error : ', mean_squared_error(y_test_pamonitor,predictions_lasso))
print(
'mean_absolute_error : ', mean_absolute_error(y_test_pamonitor_ridge,predictions_lasso))


# **Ridge regression for oxygen concentrators without pm data (only wo data )**

In [ ]:
import pandas as pd
from numpy import arange
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold

In [ ]:
Y_oxygen=dataoxyhos1['wotime_diff']
X_oxygen=dataoxyhos1.drop(['issued','Equip.reg_date','wotime_diff'],axis=1)


In [ ]:
X_train_oxy_ridge, X_test_oxy_ridge, y_train_oxy_ridge, y_test_oxy_ridge = train_test_split(X_oxygen,Y_oxygen, test_size=0.3, random_state=101)

In [ ]:
X_train_oxy_ridge.head(3)

In [ ]:
#define cross-validation method to evaluate model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [ ]:
#define model
model_ridge_oxy = RidgeCV(alphas=arange(0, 0.5, 0.01), cv=cv, scoring='neg_mean_absolute_error')

In [ ]:
#fit model
model_ridge_oxy.fit(X_train_oxy_ridge,y_train_oxy_ridge)

#display lambda that produced the lowest test MSE
print(model_ridge_oxy.alpha_)

In [ ]:
predictions_oxyridge=model_ridge_oxy.predict(X_test_oxy_ridge)

In [ ]:
# model evaluation
print(
'mean_squared_error : ', mean_squared_error(y_test_oxy_ridge,predictions_oxyridge))
print(
'mean_absolute_error : ', mean_absolute_error(y_test_oxy_ridge,predictions_oxyridge))


# **Ridge regression for oxygen concentrators with PM data 

In [ ]:
import pandas as pd
from numpy import arange
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold

In [ ]:
pm_wo_attributes_oxygen.head(3)

In [ ]:
Y_oxygen_pm=pm_wo_attributes_oxygen['wotime_diff']


In [ ]:
X_oxygen_pm=pm_wo_attributes_oxygen.drop(columns=['wotime_diff','dev','ID_y','inventarieID'])

In [ ]:
X_oxygen_pm.head(3)

In [ ]:
X_train_oxy_ridge_pm, X_test_oxy_ridge_pm, y_train_oxy_ridge_pm, y_test_oxy_ridge_pm = train_test_split(X_oxygen_pm,Y_oxygen_pm, test_size=0.3, random_state=101)

In [ ]:
X_train_oxy_ridge_pm.head(3)

In [ ]:
#define cross-validation method to evaluate model
cv4 = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [ ]:
#define model
model_ridge_oxy_pm = RidgeCV(alphas=arange(0, 1, 0.01), cv=cv, scoring='neg_mean_absolute_error')

In [ ]:
#fit model
model_ridge_oxy_pm.fit(X_train_oxy_ridge_pm,y_train_oxy_ridge_pm)

#display lambda that produced the lowest test MSE
print(model_ridge_oxy_pm.alpha_)

0.99

In [ ]:
predictions_oxyridge_pm=model_ridge_oxy_pm.predict(X_test_oxy_ridge_pm)

In [ ]:
# model evaluation
print(
'mean_squared_error : ', mean_squared_error(y_test_oxy_ridge_pm,predictions_oxyridge_pm))
print(
'mean_absolute_error : ', mean_absolute_error(y_test_oxy_ridge_pm,predictions_oxyridge_pm))


# **Ridge regression for anathesia without pm**

In [ ]:
import pandas as pd
from numpy import arange
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold

In [ ]:
Y_reghos2_ridge=data_aneshos1woeq['wotime_diff']
X_reghos2_ridge=data_aneshos1woeq.drop(['dev','ID_y','issued','Equip.reg_date','wotime_diff','inventarieID'],axis=1)


In [ ]:
X_reghos2_ridge.head(3)

In [ ]:
import matplotlib.pyplot as plt

x = data_aneshos1woeq['Equip.reg_date']
plt.hist(x)
plt.show()

In [ ]:
X_train_anathesia_ridge, X_test_anathesia_ridge, y_train_anathesia_ridge, y_test_anathesia_ridge = train_test_split(X_reghos2_ridge,Y_reghos2_ridge, test_size=0.3, random_state=101)

In [ ]:
X_train_anathesia_ridge.head(3)

In [ ]:
#define cross-validation method to evaluate model
cv5 = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [ ]:
#define model
model_ridge_ana = RidgeCV(alphas=arange(0, 1, 0.01), cv=cv, scoring='neg_mean_absolute_error')

In [ ]:
#fit model
model_ridge_ana.fit(X_train_anathesia_ridge,y_train_anathesia_ridge)

#display lambda that produced the lowest test MSE
print(model_ridge_ana.alpha_)

0.99

In [ ]:
predictions_anaridge=model_ridge_ana.predict(X_test_anathesia_ridge)

In [ ]:
# model evaluation
print(
'mean_squared_error : ', mean_squared_error(y_test_anathesia_ridge,predictions_anaridge))
print(
'mean_absolute_error : ', mean_absolute_error(y_test_anathesia_ridge,predictions_anaridge))


# **Ridge regression for anathesia with pm data included**

In [ ]:
data_anesWo_pm.head(3)

In [ ]:
data_anesWo_pm['Equip.reg_date']=(pd.to_datetime(data_anesWo_pm['Equip.reg_date']))
data_anesWo_pm['Scrapped_date']=(pd.to_datetime(data_anesWo_pm['Scrapped_date']))
data_anesWo_pm['issued']=(pd.to_datetime(data_anesWo_pm['issued']))
data_anesWo_pm['age']=(data_anesWo_pm['Scrapped_date']-data_anesWo_pm['Equip.reg_date']).dt.days
data_anesWo_pm['wotime_diff']=(data_anesWo_pm['issued']-data_anesWo_pm['Equip.reg_date']).dt.days

In [ ]:
data_anesWo_pm.shape

In [ ]:
 import matplotlib.pyplot as plt

x = data_anesWo_pm['Equip.reg_date']
plt.hist(x)
plt.show()

In [ ]:
#taking out outliers
noutlier_data_anesWo_pm=data_anesWo_pm.loc[data_anesWo_pm['Equip.reg_date']<='1990-01-01']

In [ ]:
noutlier_data_anesWo_pm.shape

In [ ]:
#let us take out all the unnecessary columns from our way 
data_anesWo_pm.drop(columns=[])

# **Decsion tree for regression compared to linear regression flavours**

# Anathesia decision tree 

In [ ]:
import pandas as pd
from numpy import arange
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold

In [ ]:
Y_reghos2_ridge=data_aneshos1woeq['wotime_diff']
X_reghos2_ridge=data_aneshos1woeq.drop(['dev','ID_y','issued','Equip.reg_date','wotime_diff','inventarieID'],axis=1)


In [ ]:
X_reghos2_ridge.head(3)

In [ ]:
import matplotlib.pyplot as plt

x = data_aneshos1woeq['Equip.reg_date']
plt.hist(x)
plt.show()

In [ ]:
X_train_anathesia_ridge, X_test_anathesia_ridge, y_train_anathesia_ridge, y_test_anathesia_ridge = train_test_split(X_reghos2_ridge,Y_reghos2_ridge, test_size=0.3, random_state=101)

In [ ]:
X_train_anathesia_ridge.head(3)

In [ ]:
from sklearn.model_selection import train_test_split # for splitting the data
from sklearn.metrics import mean_squared_error # for calculating the cost function
from sklearn.tree import DecisionTreeRegressor # for building the model

In [ ]:
# Initializing the Decision Tree Regression model
model_DT= DecisionTreeRegressor(random_state = 0)

# Fitting the Decision Tree Regression model to the data
model_DT.fit(X_train_anathesia_ridge,y_train_anathesia_ridge)

In [ ]:
# Predicting the target values of the test set
y_pred_dt = model_DT.predict(X_test_anathesia_ridge)

# RMSE (Root Mean Square Error)
rmse = float(format(np.sqrt(mean_squared_error(y_test_anathesia_ridge, y_pred_dt)), '.3f'))
print("\nRMSE: ", rmse)
RMSE: 133.351

In [ ]:
print(
'mean_absolute_error : ', mean_absolute_error(y_test_anathesia_ridge,y_pred_dt))

In [ ]:
# model evaluation
print(
'mean_squared_error : ', mean_squared_error(y_test_anathesia_ridge,y_pred_dt))


# **patient monitor**

In [ ]:
X_PA=no_outlier_pamonitor.drop(['Equip.reg_date','Scrapped_date','issued','AOEkKlarDat','Model','last_pm','first_pm','inventarieID'],axis=1)
Y_PA=no_outlier_pamonitor['wotime_diff']

In [ ]:
X_train_pamonitor_ridge, X_test_pamonitor_ridge, y_train_pamonitor_ridge, y_test_pamonitor_ridge = train_test_split(X_PA,Y_PA, test_size=0.3, random_state=101)

In [ ]:
X_train_pamonitor_ridge.head(3)

In [ ]:
 # Initializing the Decision Tree Regression model
model_pa = DecisionTreeRegressor(random_state = 0)

# Fitting the Decision Tree Regression model to the data
model_pa.fit(X_train_pamonitor_ridge,y_train_pamonitor_ridge)

In [ ]:
predictions_pa_dt=model_pa.predict(X_test_pamonitor_ridge)

In [ ]:
# model evaluation
print(
'mean_squared_error : ', mean_squared_error(y_test_pamonitor,predictions_pa_dt))
print(
'mean_absolute_error : ', mean_absolute_error(y_test_pamonitor_ridge,predictions_pa_dt))


# **OXYGEN concentrator

In [ ]:
import pandas as pd
from numpy import arange
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold

In [ ]:
pm_wo_attributes_oxygen.head(3)

In [ ]:
Y_oxygen_pm=pm_wo_attributes_oxygen['wotime_diff']


In [ ]:
X_oxygen_pm=pm_wo_attributes_oxygen.drop(columns=['wotime_diff','dev','ID_y','inventarieID'])

In [ ]:
X_oxygen_pm.head(3)

In [ ]:
X_train_oxy_ridge_pm, X_test_oxy_ridge_pm, y_train_oxy_ridge_pm, y_test_oxy_ridge_pm = train_test_split(X_oxygen_pm,Y_oxygen_pm, test_size=0.3, random_state=101)

In [ ]:
X_train_oxy_ridge_pm.head(3)

In [ ]:
# Initializing the Decision Tree Regression model
model_oxy_dt = DecisionTreeRegressor(random_state = 0)

# Fitting the Decision Tree Regression model to the data
model_oxy_dt.fit(X_train_oxy_ridge_pm, y_train_oxy_ridge_pm)

In [ ]:
predictions_oxdt_pm=model_oxy_dt.predict(X_test_oxy_ridge_pm)

In [ ]:
# model evaluation
print(
'mean_squared_error : ', mean_squared_error(y_test_oxy_ridge_pm,predictions_oxdt_pm))
print(
'mean_absolute_error : ', mean_absolute_error(y_test_oxy_ridge_pm,predictions_oxdt_pm))


# **experiment of differntiating issued dates work order 22/11/2022**

In [ ]:
data_aneshos1woeq.head(10)

will sort by inventaried id and keep it in different new data frame

In [ ]:
df_sortinver= data_aneshos1woeq.sort_values(by=['inventarieID','issued'])

In [ ]:
df_sortinver.tail(20)

In [ ]:
(df_sortinver.iloc[:,6:6:1]).shape

In [ ]:
df_sortinver.iloc[:,5:6:1]

In [ ]:
len(df_sortinver)

how can we read single value within a column?

In [ ]:
(df_sortinver.iloc[2]).Count_wo

# **Studies on preventive maintenance activities**

# **from all data patterns **

In [ ]:
all_data_together.head(3)

In [ ]:
all_data_together.shape

oxygen concentrators average preventive maintenance data 

age has strong correlation with count of work orders done , does it has a correlation with meanttime between work orders ?

there is a high correlation between count of workorders and age 
also there is high correlation between count of pm and meantime between pm
-what we are interested is finding 
1-average time between pm for oxygen concenterators and the average age for these devices?

**average meatime between pm activities for the oxygen concentrators is 5.7 months or 173 days .**

**average age for oxygen concentrators is 2329 days or 6.4 years.**

**average workorders within life time of an oxygen concentrator is 5 work orders and average meantime between workorders is 242 days or 8 months **

**we need to select all data which has it pm activities for more than 5.7 months and see if we get more work orders or not ?**

***glossary for PM-his7 data : ***

[blnRecurring] LW: Is scheduled as recurring. Meaning it after the PM is completed, another PM session will be scheduled.
      ,[intTimeInterval] LW: The period between the recurring schedules. Related to if bInRecurring is true.
      ,[PM_TimeIntervalUnitID] LW: The unit type of the period between the recurring schedules i.e. days, months. bInRecurring is true.
      ,[intTimeIntervalRecommended] LW: Is this data populated? It’s the same as [PM_TimeInterval] but what is recommend by the manufacturer.
      ,[PM_TimeIntervalUnitIDRecommended] LW: Is this data populated? It’s the same as [PM_TimeIntervalUnitID] but what is recommend by the manufacturer.
      ,[intEstimatedPMTimeHours] LW: Estimation of how long the PM will take to perform
      ,convert(date,[datLatest]) LW: Latest performed PM.
      ,convert(date,[datNext]) LW: The upcoming PM to be performed.
      ,[InventarierID] LW: Equipment ID
      ,workorderid AS [ArborderID] LW: Work Order ID
      ,[blnHiddenPMClock] LW: Not sure, could you give examples of how the data looks?
      ,[DeviatingTimeIntervalRecommended] LW: Is this data populated?
        ,DATEDIFF(DD,[datLatest],[datCreated]) AS PM_diff LW: The number of days between when the PM should have been performed and when it was performed.

In [ ]:
df8=(r"/content/drive/MyDrive/Thesis_datasets/New-data-SoftPro/Hospitals1/pm_history.csv")

In [ ]:
 data8= pd.read_csv(df8,sep=';',header=0)

In [ ]:
data8

In [ ]:
data8.isnull().sum()

In [ ]:
#dict = {'FUSenaste': 'prevPm',
       # 'FUIntNasta': 'nexPm',
        #'InkDat':'Equip.reg_date',
        #'SkrotDatum':'Scrapped_date',
        #'Benamn':'dev',
      #'Fabrikat':'Brand',
        #'Modell':'Model',
        #'intRisktal':'calc.Risk'}

In [ ]:
#fin.rename(columns=dict,inplace=True)

In [ ]:
len(data8)

**substitute scrapped null values with data collection**

In [ ]:
data8.dropna(inplace=True)

In [ ]:
data8.shape

In [ ]:
data8

loading all preventive maintenance queries again,this time we need to connect those data to specific devices so that we can relate between equipment , age and pm data using inventarierID as a reference 

---



In [ ]:
data8.set_index('InventarierID',inplace=True)

In [ ]:
data8.head(3)

In [ ]:
all_pm_fields= pd.merge(eq,data8,left_index=True, right_index=True)

In [ ]:
all_pm_fields.head(5)

In [ ]:
all_pm_fields.shape

In [ ]:
oxygen=all_pm_fields.query("dev=='Anestesiarbetsstation'")



In [ ]:
oxygen